# Spark DataFrame Basics - Tutorial 1 
## Purpose: Processing a JSON file
### Robert Esteves
### 2018-10-27

In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [4]:
source_dataset = '/home/robert/Downloads/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json'

In [5]:
df = spark.read.json(source_dataset)

In [6]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [7]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.columns

['age', 'name']

In [9]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [10]:
# Statistical Summary of the numerical columns
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [11]:
# How to clarify the schema to optimze the dataframe schema

In [12]:
from pyspark.sql.types import (StructField, StringType, 
                               IntegerType, StructType)

In [13]:
# Define Field Structure: <field name>, <data type>, <allow nulls>

data_schema = [
    StructField('age', IntegerType(), True),
    StructField('name', StringType(), True)              
              ]

In [14]:
final_structure = StructType(fields = data_schema)

In [15]:
df = spark.read.json(source_dataset, schema = final_structure)

In [16]:
df

DataFrame[age: int, name: string]

In [17]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [18]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



## How to extract data from a data frame?

In [19]:
df['age']

Column<b'age'>

In [20]:
type(df['age'])

pyspark.sql.column.Column

### To extract data from a data frame use the SELECT method

In [21]:
df.select('age')  # This method returns a column

DataFrame[age: int]

### To display the the data use the .show() method

In [25]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [24]:
df.select('age').show() # This method returns a dataframe that contains a single column. You also get more flexebility

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



## To view the first two rows of the Data Frame use the head method

In [27]:
df.head(2) # this command returns a list

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [31]:
df.head(2)[0]  # returning row 0 object

Row(age=None, name='Michael')

In [30]:
type(df.head(2)[0])

pyspark.sql.types.Row

### To select several columns use a list with the select statment

In [32]:
df.select(['age', 'name'])

DataFrame[age: int, name: string]

In [33]:
type(df.select(['age', 'name']))

pyspark.sql.dataframe.DataFrame

In [34]:
df.select(['age', 'name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### How to create a new column in the data frame

In [35]:
df.withColumn('new_age', df['age']).show()

+----+-------+-------+
| age|   name|new_age|
+----+-------+-------+
|null|Michael|   null|
|  30|   Andy|     30|
|  19| Justin|     19|
+----+-------+-------+



In [36]:
df.withColumn('double_age', df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



### Understand that these changes are not permanent. To make the changes perment, you have to save the changes to a new variable (data frame)

In [37]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### How to rename a column?

In [40]:
df.withColumnRenamed('age', 'my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



### How to use PURE SQL to directly interact with the data frame?
- You must create a view
 - This means that you must register the dataframe as a SQL temporary view
 - Use createOrReplaceTempView
 - Give the view a name such as vw_people

In [44]:
df.createOrReplaceTempView('vw_people')

In [45]:
results = spark.sql("SELECT * FROM vw_people")

In [46]:
results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [48]:
new_results = spark.sql("SELECT * FROM vw_people WHERE age > 20")

In [49]:
new_results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



## Note: do not use the semicololn (;) to end your SQL statement, because you will get an error.